# Preprocessing of scraped recipes

### Import libraries

In [1]:
import pandas as pd
import re
import pickle
import numpy as np
pd.set_option('display.max_colwidth', None)
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')
#import en_core_web_sm
stop_words = set([w.lower() for w in list(STOP_WORDS)])
nlp.Defaults.stop_words |= {" f "," s "," etc"}
import tensorflow as tf


### Load the data

In [3]:
json_file = 'new_recipe.pkl'
set_json = pickle.load(open(json_file, 'rb'))
#print (set_json[: , 1])
print(type(set_json))

<class 'collections.defaultdict'>


In [4]:
filename='new_recipe.pkl'
df = pd.DataFrame(pd.read_pickle(filename))
df.head(3)

,Recipe,INSTRUCTIONS
0,"[½ cup raw cashews*, 2 tablespoons nutritional yeast, ½ teaspoon sea salt, ½ teaspoon lemon zest]","Place the cashews, nutritional yeast, salt, and lemon zest in a small food processor. Pulse until the mixture resembles a coarse meal and has a soft, crumbly texture.\n\nStore in the fridge for up to 5 days, freeze after that."
1,"[4 cups cherry tomatoes, (half for roasting, half raw), Extra-virgin olive oil, for drizzling, 1 cup dry Israeli couscous, ¼ teaspoon smoked paprika, 1 tablespoon lemon juice, more if desired, 1 garlic clove, minced, Leaves from 6 sprigs fresh thyme, more for garnish, 1½ cups roasted chickpeas, ¼ cup fresh basil, more for garnish, 2 Persian cucumbers, thinly sliced, ⅓ cup crumbled feta cheese, Sea salt and freshly ground black pepper]","Roast 2 cups of the cherry tomatoes according to this recipe. These can be made in advance and stored in the fridge for a few days.\n\nCook the couscous according to the package directions until al dente, about 9 minutes. Drain and set aside to cool.\n\nIn the bottom of a large bowl, combine 1 tablespoon of olive oil, the lemon juice, garlic, thyme, ¼ teaspoon salt, and several grinds of black pepper. Add the cooled couscous and toss. Slice the remaining raw cherry tomatoes in half and add them to the bowl with the roasted tomatoes, the chickpeas, basil, cucumbers, and feta. Top with more fresh herbs and a generous drizzle of olive oil. Season to taste."
2,"[1 ½ cups frozen blueberries, 3 tablespoons whole rolled oats, 3 soft medjool dates, Handful of fresh spinach, ½ tablespoon fresh lemon juice, ¼ teaspoon cinnamon, Pinch of salt, 1 tablespoon Whole Foods Market™ Plant-Based Fit Protein, 6 ice cubes, ¾ to 1 cup almond milk or light coconut milk, or as needed to blend, optional garnishes: fresh blueberries, oats, crushed almonds, 2 peaches, sliced and frozen, 2 heaping tablespoons pecans, 2 soft medjool dates, ¼ teaspoon cinnamon, 1 to 2 tablespoons Whole Foods Market™ Plant-Based Fit Protein, ¼ cup light canned coconut milk, or additional almond milk, ¾ to 1 cup almond milk, or as needed to blend, optional garnishes: coconut cream, sliced fresh peaches, crushed pecans, 2 cups frozen strawberries, plus 2 fresh strawberries, sliced for garnish, Small handful of fresh spinach, 1 tablespoon chia seeds, 3 soft medjool dates, 2 tablespoons peanut or almond butter, 2 to 3 tablespoons 365 Everyday Value® Pea Protein, 1¼ cups almond milk, more as needed to blend, optional garnishes: sliced strawberry, drizzle of peanut butter, chia seeds, 4 frozen coconut milk ice cubes, 1½ cup frozen spinach leaves, ¼ cup mint leaves, frozen, 3 soft medjool dates, 1½ tablespoon cacao nibs, 3 tablespoons 365 Everyday Value® Pea Protein, 1 to 1½ cup almond milk, or as needed to blend, optional garnishes: coconut cream, mint leaves, cacao nibs]","For each smoothie - blend ingredients in a blender. Use the blender's baton (if you have one) to help. Add more almond milk as needed. The key to a thick smoothie is to add enough liquid to move your blender blade, but not too much to make the smoothie thin and watery."


### Check the shape and info

In [5]:
len(set_json)

2

In [6]:
type(set_json)
set_json.items()
set_json.keys()
set_json["Recipe"]
#set_json.values()
#print (set_json)
#print (set_json["Recipe"])
print(len(set_json))

2


In [7]:
df.shape

(1009, 2)

<b><br> We have 1035 rows and 2 columns.
<br>`Recipe` and `INSTRUCTIONS` are our 2 columns.
<br> Also our present columns will be our labels for each sentence.
<br> Later we will split the data for sentences:
<br>each row will be a sentence and we will have a label 1- recipe and 0 - instructions.</b>

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Recipe        1009 non-null   object
 1   INSTRUCTIONS  1009 non-null   object
dtypes: object(2)
memory usage: 15.9+ KB


<b>Ok, good. <br>Now let's transform our first column `Recipe` :</b>

In [10]:
def from_list_to_str(series):
    return ' '.join([words for words in series])


recipe_col = df["Recipe"]
recipe = pd.DataFrame(recipe_col.apply(from_list_to_str).to_numpy(), columns=['sentence'])
recipe.head()

,sentence
0,½ cup raw cashews* 2 tablespoons nutritional yeast ½ teaspoon sea salt ½ teaspoon lemon zest
1,"4 cups cherry tomatoes, (half for roasting, half raw) Extra-virgin olive oil, for drizzling 1 cup dry Israeli couscous ¼ teaspoon smoked paprika 1 tablespoon lemon juice, more if desired 1 garlic clove, minced Leaves from 6 sprigs fresh thyme, more for garnish 1½ cups roasted chickpeas ¼ cup fresh basil, more for garnish 2 Persian cucumbers, thinly sliced ⅓ cup crumbled feta cheese Sea salt and freshly ground black pepper"
2,"1 ½ cups frozen blueberries 3 tablespoons whole rolled oats 3 soft medjool dates Handful of fresh spinach ½ tablespoon fresh lemon juice ¼ teaspoon cinnamon Pinch of salt 1 tablespoon Whole Foods Market™ Plant-Based Fit Protein 6 ice cubes ¾ to 1 cup almond milk or light coconut milk, or as needed to blend optional garnishes: fresh blueberries, oats, crushed almonds 2 peaches, sliced and frozen 2 heaping tablespoons pecans 2 soft medjool dates ¼ teaspoon cinnamon 1 to 2 tablespoons Whole Foods Market™ Plant-Based Fit Protein ¼ cup light canned coconut milk, or additional almond milk ¾ to 1 cup almond milk, or as needed to blend optional garnishes: coconut cream, sliced fresh peaches, crushed pecans 2 cups frozen strawberries, plus 2 fresh strawberries, sliced for garnish Small handful of fresh spinach 1 tablespoon chia seeds 3 soft medjool dates 2 tablespoons peanut or almond butter 2 to 3 tablespoons 365 Everyday Value® Pea Protein 1¼ cups almond milk, more as needed to blend optional garnishes: sliced strawberry, drizzle of peanut butter, chia seeds 4 frozen coconut milk ice cubes 1½ cup frozen spinach leaves ¼ cup mint leaves, frozen 3 soft medjool dates 1½ tablespoon cacao nibs 3 tablespoons 365 Everyday Value® Pea Protein 1 to 1½ cup almond milk, or as needed to blend optional garnishes: coconut cream, mint leaves, cacao nibs"
3,"1 tablespoon olive oil, plus more for drizzling 1 small yellow onion, chopped 2 big garlic cloves, peeled 1 teaspoon sweet or smoked paprika ½ tablespoon balsamic vinegar, more to taste 1 14-oz. can of diced tomatoes a few pinches of sugar, optional 3 sprigs fresh thyme leaves 1 cup cooked chickpeas, drained and rinsed, plus extra for garnish 1 cup coconut milk, full fat or light 1.5 cups water ¼ cup parmesan cheese, very optional, skip if vegan red pepper flakes, optional for garnish Sea salt and freshly ground black pepper"
4,"½ cup pepitas ½ cup mint, more for garnish ½ cup parsley ½ cup thawed frozen peas, more for garnish 1 small garlic clove 2 tablespoons lemon juice ¼ teaspoon salt ¼ cup extra-virgin olive oil, more as desired 4 slices whole grain bread, toasted 6 to 8 thick asparagus spears, peeled into ribbons thinly sliced raw or pickled radishes a sprinkle of crumbled feta cheese 2 soft boiled eggs, optional* microgreens, optional"


<b>We have to add a column `label`:</b>

In [11]:
recipe['label'] = np.ones(len(recipe),int)
recipe.head()

,sentence,label
0,½ cup raw cashews* 2 tablespoons nutritional yeast ½ teaspoon sea salt ½ teaspoon lemon zest,1
1,"4 cups cherry tomatoes, (half for roasting, half raw) Extra-virgin olive oil, for drizzling 1 cup dry Israeli couscous ¼ teaspoon smoked paprika 1 tablespoon lemon juice, more if desired 1 garlic clove, minced Leaves from 6 sprigs fresh thyme, more for garnish 1½ cups roasted chickpeas ¼ cup fresh basil, more for garnish 2 Persian cucumbers, thinly sliced ⅓ cup crumbled feta cheese Sea salt and freshly ground black pepper",1
2,"1 ½ cups frozen blueberries 3 tablespoons whole rolled oats 3 soft medjool dates Handful of fresh spinach ½ tablespoon fresh lemon juice ¼ teaspoon cinnamon Pinch of salt 1 tablespoon Whole Foods Market™ Plant-Based Fit Protein 6 ice cubes ¾ to 1 cup almond milk or light coconut milk, or as needed to blend optional garnishes: fresh blueberries, oats, crushed almonds 2 peaches, sliced and frozen 2 heaping tablespoons pecans 2 soft medjool dates ¼ teaspoon cinnamon 1 to 2 tablespoons Whole Foods Market™ Plant-Based Fit Protein ¼ cup light canned coconut milk, or additional almond milk ¾ to 1 cup almond milk, or as needed to blend optional garnishes: coconut cream, sliced fresh peaches, crushed pecans 2 cups frozen strawberries, plus 2 fresh strawberries, sliced for garnish Small handful of fresh spinach 1 tablespoon chia seeds 3 soft medjool dates 2 tablespoons peanut or almond butter 2 to 3 tablespoons 365 Everyday Value® Pea Protein 1¼ cups almond milk, more as needed to blend optional garnishes: sliced strawberry, drizzle of peanut butter, chia seeds 4 frozen coconut milk ice cubes 1½ cup frozen spinach leaves ¼ cup mint leaves, frozen 3 soft medjool dates 1½ tablespoon cacao nibs 3 tablespoons 365 Everyday Value® Pea Protein 1 to 1½ cup almond milk, or as needed to blend optional garnishes: coconut cream, mint leaves, cacao nibs",1
3,"1 tablespoon olive oil, plus more for drizzling 1 small yellow onion, chopped 2 big garlic cloves, peeled 1 teaspoon sweet or smoked paprika ½ tablespoon balsamic vinegar, more to taste 1 14-oz. can of diced tomatoes a few pinches of sugar, optional 3 sprigs fresh thyme leaves 1 cup cooked chickpeas, drained and rinsed, plus extra for garnish 1 cup coconut milk, full fat or light 1.5 cups water ¼ cup parmesan cheese, very optional, skip if vegan red pepper flakes, optional for garnish Sea salt and freshly ground black pepper",1
4,"½ cup pepitas ½ cup mint, more for garnish ½ cup parsley ½ cup thawed frozen peas, more for garnish 1 small garlic clove 2 tablespoons lemon juice ¼ teaspoon salt ¼ cup extra-virgin olive oil, more as desired 4 slices whole grain bread, toasted 6 to 8 thick asparagus spears, peeled into ribbons thinly sliced raw or pickled radishes a sprinkle of crumbled feta cheese 2 soft boiled eggs, optional* microgreens, optional",1


In [12]:
recipe.to_numpy().shape
recipe
df["INSTRUCTIONS"]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

<b>Great! It's time to prepare the `INSTRUCTIONS` column</b>

In [13]:
instr_col = df["INSTRUCTIONS"].str.split('\n\n').to_numpy()
print(instr_col)
instr = np.concatenate(instr_col).reshape(-1,1)
instr = pd.DataFrame(instr, columns=['sentence'])
instr.head()

[list(['Place the cashews, nutritional yeast, salt, and lemon zest in a small food processor. Pulse until the mixture resembles a coarse meal and has a soft, crumbly texture.', 'Store in the fridge for up to 5 days, freeze after that.'])
 list(['Roast 2 cups of the cherry tomatoes according to this recipe. These can be made in advance and stored in the fridge for a few days.', 'Cook the couscous according to the package directions until al dente, about 9 minutes. Drain and set aside to cool.', 'In the bottom of a large bowl, combine 1 tablespoon of olive oil, the lemon juice, garlic, thyme, ¼ teaspoon salt, and several grinds of black pepper. Add the cooled couscous and toss. Slice the remaining raw cherry tomatoes in half and add them to the bowl with the roasted tomatoes, the chickpeas, basil, cucumbers, and feta. Top with more fresh herbs and a generous drizzle of olive oil. Season to taste.'])
 list(["For each smoothie - blend ingredients in a blender. Use the blender's baton (if y

,sentence
0,"Place the cashews, nutritional yeast, salt, and lemon zest in a small food processor. Pulse until the mixture resembles a coarse meal and has a soft, crumbly texture."
1,"Store in the fridge for up to 5 days, freeze after that."
2,Roast 2 cups of the cherry tomatoes according to this recipe. These can be made in advance and stored in the fridge for a few days.
3,"Cook the couscous according to the package directions until al dente, about 9 minutes. Drain and set aside to cool."
4,"In the bottom of a large bowl, combine 1 tablespoon of olive oil, the lemon juice, garlic, thyme, ¼ teaspoon salt, and several grinds of black pepper. Add the cooled couscous and toss. Slice the remaining raw cherry tomatoes in half and add them to the bowl with the roasted tomatoes, the chickpeas, basil, cucumbers, and feta. Top with more fresh herbs and a generous drizzle of olive oil. Season to taste."


<b>Adding column `label` with value 0</b>

In [14]:
instr['label'] = np.zeros(len(instr),int)
instr

,sentence,label
0,"Place the cashews, nutritional yeast, salt, and lemon zest in a small food processor. Pulse until the mixture resembles a coarse meal and has a soft, crumbly texture.",0
1,"Store in the fridge for up to 5 days, freeze after that.",0
2,Roast 2 cups of the cherry tomatoes according to this recipe. These can be made in advance and stored in the fridge for a few days.,0
3,"Cook the couscous according to the package directions until al dente, about 9 minutes. Drain and set aside to cool.",0
4,"In the bottom of a large bowl, combine 1 tablespoon of olive oil, the lemon juice, garlic, thyme, ¼ teaspoon salt, and several grinds of black pepper. Add the cooled couscous and toss. Slice the remaining raw cherry tomatoes in half and add them to the bowl with the roasted tomatoes, the chickpeas, basil, cucumbers, and feta. Top with more fresh herbs and a generous drizzle of olive oil. Season to taste.",0
...,...,...
4130,Season fish with a little salt & pepper on both sides.,0
4131,"Place bok choy, mushrooms, fish, scallions and pour on about ONE QUARTER of the sauce. Top with orange slices, set aside and prepare your second piece of fish the same way.",0
4132,"Wrap up your parchment by folding the left and right of the paper toward the center, creasing the edges shut as tightly as you can. Next fold and crease the top and bottom edges toward the center. (The first fold will want to fall apart until you get the top and bottom folded).",0
4133,"Bake for about 10 minutes. Remove from the oven and Let it sit for 30 seconds to 1 minute. Open it and check for done-ness. It should be opaque on the outside and still slightly translucent in the center. If your fish is not all the way cooked, fold the paper shut again and let it sit and steam for another minute or two. (no need to put it back in the oven unless it’s extremely underdone).",0


In [15]:
instr.to_numpy().shape

(4135, 2)

In [16]:
data = pd.DataFrame(np.concatenate((instr.to_numpy(), recipe.to_numpy())), columns=['paragraph','label'])
data.head()

,paragraph,label
0,"Place the cashews, nutritional yeast, salt, and lemon zest in a small food processor. Pulse until the mixture resembles a coarse meal and has a soft, crumbly texture.",0
1,"Store in the fridge for up to 5 days, freeze after that.",0
2,Roast 2 cups of the cherry tomatoes according to this recipe. These can be made in advance and stored in the fridge for a few days.,0
3,"Cook the couscous according to the package directions until al dente, about 9 minutes. Drain and set aside to cool.",0
4,"In the bottom of a large bowl, combine 1 tablespoon of olive oil, the lemon juice, garlic, thyme, ¼ teaspoon salt, and several grinds of black pepper. Add the cooled couscous and toss. Slice the remaining raw cherry tomatoes in half and add them to the bowl with the roasted tomatoes, the chickpeas, basil, cucumbers, and feta. Top with more fresh herbs and a generous drizzle of olive oil. Season to taste.",0


In [17]:
data.shape

(5144, 2)

In [18]:
data.label[data.label == 1].count()

1009

In [19]:
data.label[data.label == 0].count()

4135

In [20]:
data.isna().sum()

paragraph    0
label        0
dtype: int64

##### no null - good, check the proportion

In [21]:
round(data['label'].value_counts()/len(data)*100,2)

0    80.38
1    19.62
Name: label, dtype: float64

##### drop duplicated and empty rows

In [22]:
data[data.paragraph == '']

,paragraph,label
398,,0
802,,0
987,,0
1988,,0
2240,,0
2277,,0
2629,,0
2857,,0
3415,,0
4236,,1


In [ ]:
data = data[data.paragraph != '']
data.shape

In [23]:
data = data.drop_duplicates().reset_index(drop=True)
data.tail(50)

,paragraph,label
4823,"½ cup extra-virgin olive oil zest of 1 small lemon ½ cup extra-virgin olive oil 1½ tablespoons red chile flakes 1 cup whole rolled oats* ⅓ cup chopped pistachios 1 teaspoon cinnamon ½ teaspoon sea salt 1 tablespoon coconut oil 3 tablespoons maple syrup ¼ cup coconut flakes ⅓ cup dried goji berries ½ cup creamy almond butter ½ cup chopped dark chocolate, 50% cacao* 1½ tablespoons coconut oil",1
4824,"2 avocados 1/2 lemon 8 slices toasted bread 3 heirloom tomatoes, sliced Butter lettuce, arugula or microgreens Sea salt and freshly ground black pepper sprinkle of hemp seeds, optional",1
4825,"1 can of full fat coconut milk, refrigerated overnight ¼ cup powdered sugar (or sweetener of your choice) pinch of salt ½ teaspoon vanilla ½ teaspoon lemon oil (optional, but delicious) a few cups of mixed fresh berries a few teaspoons granulated sugar ⅛ teaspoon vanilla bean paste (scraped from 1 pod), optional* pinch of salt chopped & toasted pecans, or this no-bake crumble cinnamon, for dusting",1
4826,"2 chipotle peppers + 2 tablespoons of sauce from a can of chipotles in adobo sauce ½ cup ketchup ¼ cup apple cider vinegar 2 garlic cloves 1 teaspoon Dijon mustard ½ teaspoon smoked paprika ½ teaspoon cumin Freshly ground black pepper 1 tablespoon extra-virgin olive oil 1 small yellow onion, thinly sliced ¼ teaspoon sea salt 1 (20-ounce) can un-ripe jackfruit, drained ½ cup water 2 cups shredded cabbage 1/4 cup chopped cilantro ½ tablespoon lime juice ½ teaspoon olive oil Sea salt and freshly ground black pepper 8 slider buns or 6 hamburger buns Desired fixings: pickles, mustard, serrano peppers, etc.",1
4827,"1 yellow squash 1 zucchini 8 ounces cremini mushrooms, stemmed 1 small red onion 1 red bell pepper 1 green bell pepper 1 ear fresh corn, cut into 1-inch rounds Extra virgin olive oil, for drizzling Sea salt and freshly ground black pepper tzatziki, pesto, or Greek dressing for drizzling/serving",1
4828,"1 red pepper, diced 1 ripe mango, cubed 1 small cucumber, chopped (1½ cups) ½ small red onion, diced (1/3 cup) 1 small garlic clove, minced 1 (14-ounce) can chickpeas, drained and rinsed ½ chopped fresh herbs (mint, basil, and/or cilantro) ½ cup uncooked orzo Extra-virgin olive oil, for drizzling 1 packed cup arugula ¼ cup toasted pine nuts 2 tablespoons tahini 2 tablespoons extra-virgin olive oil 2 tablespoons fresh lemon juice 1 tablespoon sherry or white wine vinegar ½ teaspoon cumin ½ teaspoon sweet paprika 2 tablespoons water ½ teaspoon sea salt, more to taste freshly ground black pepper",1
4829,"½ cup walnuts 3-4 tablespoons brown sugar 1 teaspoon cinnamon tiny pinch of salt 2 tablespoons cold butter (I use vegan earth balance) a few teaspoons flour*, if necessary for the crumble 4-6 peaches ice cream (I used coconut, use what you like)",1
4830,"sourdough bread slices grated fresh parmesan cheese (or blend of parmesan & white cheddar) fig jam a few sprigs of arugula, per sandwich butter or olive oil, for the pan optional: drizzle of balsamic vinegar",1
4831,"2 tablespoons extra-virgin olive oil 1 cup chopped yellow onion 1 red bell pepper, seeded and diced ¼ teaspoon sea salt, more to taste Freshly ground black pepper 3 medium garlic cloves, minced ½ teaspoon smoked paprika ½ teaspoon ground cumin Pinch of cayenne pepper, optional 1 28-ounce can crushed tomatoes 2 tablespoons harissa paste*, see note 1 cup fresh spinach, chopped 3 to 5 eggs ⅓ cup crumbled feta cheese ¼ cup fresh parsley leaves 1 avocado, diced Microgreens for garnish, optional Toasted bread, for serving",1
4832,"1 medium-sized cucumber, peeled only if you prefer 2 tablespoons rice vinegar ½ teaspoon agave syrup or sugar 1 teaspoon sesame seeds Sea salt 1 teaspoon wakame flakes (or arame, or hiziki) (optional) ½ teaspoon sesame oil (optional)",1


##### check the proportion again

In [24]:
round(data['label'].value_counts()/len(data)*100,2)

0    79.62
1    20.38
Name: label, dtype: float64

##### Chenged not much, but +6.2%  of sentences in  `ingredients` and -6.2% in `recipe` after droping duplacates
##### We have a little bit imballanced data set
##### Replace `.` to token `zzdot` and split on it

In [25]:
DIGIT_RX = "(\d+([\.|,]\d+)?[\w]*[\s|-])|[^A-Za-z\,()\.'\-: ]{1,7}"
SYMBOL_RX ="[/(/)\-/*/,]|[^ -~]"
DOT_RX = "\.|\:"

def replace_numbers_str(series):
        """
        replace numbers expression (11 ; 11,00;  1111.99; 23-th 25-,1/2,¼) with tag ' zNUM ',
        . and : replace with ' zDOT '
        :param series: pandas series of text strings
        :return: series with replaced numbers
        """
        new_series1 = re.sub(DIGIT_RX," zNUM ",series)
        new_series2 = re.sub(SYMBOL_RX,' ',new_series1)
        new_series = re.sub(DOT_RX," zDOT",new_series2)
        return new_series

data['paragraph'] = data.paragraph.apply(replace_numbers_str)
data.tail(15)

,paragraph,label
4858,zNUM cup pumpkin puree zNUM eggs zNUM cup almond milk zNUM cup olive oil or any neutral oil zNUM cup maple syrup zNUM tablespoon apple cider vinegar zNUM teaspoon vanilla zNUM cup all purpose flour zNUM cup whole wheat flour zNUM tablespoon pumpkin pie spice zNUM teaspoon baking powder zNUM teaspoon baking soda zNUM teaspoon sea salt,1
4859,zNUM small yellow onion sliced into wedges zNUM garlic clove unpeeled zNUM small or zNUM large cauliflower florets and core roughly chopped zNUM pound zNUM cups heated vegetable broth zNUM tablespoons extra virgin olive oil more for drizzling zNUM tablespoon fresh lemon juice zNUM to zNUM teaspoon curry powder pinch of cayenne pepper more if desired sea salt and freshly ground black pepper Creamy Kale Pepita Pesto from the grain bowl recipe microgreens optional red pepper flakes optional naan bread for serving,1
4860,zNUM delicata squash halved lengthwise and seeded sliced into zNUM inch segments zNUM small red onion sliced into wedges zNUM zNUM ounce can chickpeas drained and rinsed zNUM whole garlic cloves zNUM tablespoon extra virgin olive oil plus more for drizzling zNUM tablespoon fresh lemon juice Sea salt and freshly ground black pepper about zNUM leaves of lacinato kale thinly sliced zNUM cup cooked farro zNUM see note zNUM gala apple diced zNUM cup dried cranberries zNUM cup pepitas zNUM cup tahini zNUM tablespoon apple cider vinegar zNUM teaspoon maple syrup zNUM tablespoons warm water more as needed sea salt and freshly ground black pepper,1
4861,Baking chocolate I used this zNUM oz zNUM cacao baking bar zNUM teaspoons coconut oil zNUM teaspoons maple syrup zNUM tablespoons crushed pistachios zNUM tablespoons crushed pecans zNUM tablespoons toasted coconut flakes zNUM tablespoons dried cranberries and a few pinches of coarse sea salt,1
4862,slices of toasted baguette mascarpone cheese see vegan version in the notes blackberries macerated w zNUM a pinch of sugar and salt lemon zest a few thyme leaves drizzle of honey optional more salt to taste,1
4863,zNUM cup hazelnuts zNUM cup pistachios zNUM tablespoon whole dried coriander zNUM tablespoon sesame seeds zNUM teaspoon dried peppercorns zNUM teaspoon dried orange peel zNUM teaspoon dried cilantro zNUM teaspoon salt Greek yogurt Extra virgin olive oil Squeeze of lemon Pinches of salt,1
4864,zNUM tablespoons extra virgin olive oil zNUM ounces cremini mushrooms de stemmed and sliced zNUM ounces Spaghetti plus zNUM cup reserved pasta water zNUM garlic cloves minced zNUM cup dry white wine such as chardonnay or pinot grigio zNUM to zNUM cup cashew cream below zNUM cup chopped fresh tarragon zNUM cup chopped fresh chives zNUM cup shaved Parmigiano Reggiano cheese Sea salt and freshly ground black pepper zNUM cup raw cashews zNUM zNUM cup water or skip this mixture and use regular cream zNUM clove garlic a small one zNUM teaspoon fresh lemon juice zNUM teaspoon dijon mustard zNUM teaspoon sea salt,1
4865,zNUM cups fresh cilantro zNUM garlic clove zNUM cup lime juice zNUM teaspoons maple syrup or honey zNUM teaspoon ground coriander zNUM teaspoon sea salt zNUM cup extra virgin olive oil zNUM avocado or zNUM cup plain whole milk Greek yogurt,1
4866,zNUM ounces extra firm tofu cut into cubes zNUM daikon at least zNUM diameter and about zNUM long zNUM medium cucumber zNUM carrots peeled into ribbons zNUM radishes thinly sliced zNUM avocado diced zNUM cup cilantro zNUM cup mint leaves zNUM scallions thinly sliced zNUM tablespoons toasted and chopped cashews extra virgin olive oil sea salt sriracha lime wedges for serving zNUM tablespoons tamari zNUM small garlic cloves minced zNUM teaspoons fresh lime juice zNUM teaspoons rice vinegar zNUM tablespoon cane sugar or maple or agave zNUM cup water zNUM tablespoons creamy cashew butter or peanut butter,1
4867,zNUM cup brown rice flour zNUM cup almond flour zNUM cup hemp seeds zNUM tablespoons ground flaxseed meal zNUM tablespoons sesame seeds zNUM teaspoon baking powder zNUM 

# Uzing numpy + pandas in one step all notebook

In [27]:
def from_list_to_str(series):
    """Function to transform series of list(str) to series of string
    param: series:list(str)
    return: new series: str
    """
    return ' '.join([words for words in series])

def stratified_split_data(text, label, test_size):
    """
    The function shuffles and splits data set into Train and Test sets stratified on label
    :param text: series
    :param label: series
    :param test_size: float
    :return: (train_data, teat_data): list(ndArray,ndArray)
    source link: https://stackoverflow.com/questions/57792113/stratify-batch-in-tensorflow-2
                 https://www.tensorflow.org/api_docs/python/tf/data/Dataset
    """
    TEST_SIZE = test_size
    DATA_SIZE = len(text)

    # Create data
    X_data = text
    y_data = label
    samples1 = np.sum(y_data)
#     logging.info(f'Percentage of 1 = {samples1 / len(y_data)}')
    print('Percentage of 1 = ', samples1 / len(y_data))

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data))

    # Gather data with 0 and 1 labels separately
    class0_dataset = dataset.filter(lambda x, y: y == 0)
    class1_dataset = dataset.filter(lambda x, y: y == 1)

    # Shuffle them
    class0_dataset = class0_dataset.shuffle(DATA_SIZE)
    class1_dataset = class1_dataset.shuffle(DATA_SIZE)

    # Split them
    class0_test_samples_len = int((DATA_SIZE - samples1) * TEST_SIZE)
    class0_test = class0_dataset.take(class0_test_samples_len)
    class0_train = class0_dataset.skip(class0_test_samples_len)

    class1_test_samples_len = int(samples1 * TEST_SIZE)
    class1_test = class1_dataset.take(class1_test_samples_len)
    class1_train = class1_dataset.skip(class1_test_samples_len)

    print('Train Class 0 = ', len(list(class0_train)), ' Class 1 = ', len(list(class1_train)))
    print('Test Class 0 = ', len(list(class0_test)), ' Class 1 = ', len(list(class1_test)))
#     logging.info(f'Train Class 0 = {len(list(class0_train))} Class 1 = {len(list(class1_train))}')
#     logging.info(f'Test Class 0 = {len(list(class0_test))} Class 1 = {len(list(class1_test))}')

    # Gather datasets
    train_dataset = class0_train.concatenate(class1_train).shuffle(DATA_SIZE)
    test_dataset = class0_test.concatenate(class1_test).shuffle(DATA_SIZE)

    print('Train dataset size = ', len(list(train_dataset)))
    print('Test dataset size = ', len(list(test_dataset)))
#     logging.info(f'Train dataset size = {len(list(train_dataset))}')
#     logging.info(f'Test dataset size = {len(list(test_dataset))}')

    return train_dataset, test_dataset


# load data
filename='new_recipe.pkl'
df = pd.DataFrame(pd.read_pickle(filename))
recipe_col = df["Recipe"].apply(from_list_to_str).to_numpy()

# #transform recioe to array and give a label 1
recipe = recipe_col.reshape(-1,1)
recipe = np.hstack((recipe,np.ones(len(recipe),int).reshape(-1,1)))
print(recipe.shape)

# #transform instructions to array and give a label 0
instr_col = df["INSTRUCTIONS"].str.split('\n\n').to_numpy()
instr = np.concatenate(instr_col).reshape(-1,1)
instr = np.hstack((instr,np.zeros(len(instr),int).reshape(-1,1)))
print(instr.shape)

# #forming a full data array with labels
data = np.concatenate((instr, recipe), axis=0)
print('shape all of paragraphs data matrix',data.shape)


# # remove duplicates
unique = np.unique(data.astype(str), axis=0)
print('shape without duplicates',unique.shape)
# #remove empty string rows(from table)
unique = np.delete(unique.astype(str),np.where(unique == ''),axis=0)
print('shape without empty string rows',unique.shape)
# pd.DataFrame(unique)[4075:]



(1009, 2)
(4135, 2)
shape all of paragraphs data matrix (5144, 2)
shape without duplicates (4873, 2)
shape without empty string rows (4871, 2)


In [28]:

text = pd.DataFrame(unique)[0]
label = pd.DataFrame(unique)[1].apply(int)
test_size = 0.2
train_dataset, test_dataset = stratified_split_data(text, label, test_size)

Percentage of 1 =  0.2036542804352289
Train Class 0 =  3104  Class 1 =  794
Test Class 0 =  775  Class 1 =  198
Train dataset size =  3898
Test dataset size =  973


In [29]:
for text_batch, label_batch in train_dataset.take(5):
    print("paragraph", text_batch.numpy())
    print("label", label_batch.numpy())

paragraph b'Sesame Tofu: Slice your tofu and gently pat it with a clean towel to dab up some of the excess water. In an 8\xc3\x978 square baking pan (with edges), mix together the soy sauce, rice vinegar, and sesame oil. Place the tofu slabs into the pan and flip them so you coat both sides. Set aside and let it marinate while you finish everything else (15-20 minutes or so). After it\xe2\x80\x99s marinated, sprinkle both sides of each with some sesame seeds and some panko bread crumbs. Heat a large skillet with some olive oil. When the oil is hot, cook tofu for 2-3 minutes per side. Let cool to room temp and chop into small cubes for the salad.'
label 0
paragraph b'Make the dressing: Pulse the scallion, sesame oil, lemongrass, cilantro, basil, garlic, soy sauce, coconut milk, agave, sriracha, and lime juice and zest in a food processor until combined but not pureed, so you can see course flecks of herbs. Taste and adjust seasonings.'
label 0
paragraph b'Thinly slice the radish into ro

In [30]:
#constants regex
DIGIT_RX = "(\d+([\.|,]\d+)?[\w]*[\s|-])|[^A-Za-z\,()\.'\-: ]{1,7}"
SIMBOL_RX ="[/(/)\-/*/,]|[^ -~]"
DOT_RX = "\.{1,4}|\:"


def replace_numbers_str(series):
        """
        replace numbers expression (11 ; 11,00;  1111.99; 23-th 25-,1/2,¼) with tag ' zNUM ',
        . and : replace with ' zDOT '
        :param series: pandas series of text strings
        :return: series with replaced numbers
        """
        new_series1 = re.sub(DIGIT_RX," zNUM ",series)
        new_series2 = re.sub(SIMBOL_RX,' ',new_series1)
        new_series = re.sub(DOT_RX," zDOT ",new_series2)
        return new_series


def lemmatiz(series):
    """Transform all words to lemma, add tag  -PRON-
    paramLseries:str
    return: series:str"""
    new_series =  ' '.join([word.lemma_ for word in nlp(series)])
    return new_series

def have_pron(series):
    """Give the answer is there a pron in the paragraph
    param:series:str
    return: int 1 or 0"""
    answer = 0
    if series.__contains__('-PRON-'):
        answer = 1
    return answer


def remove_punctuation(series):
    """ Remove punctuation from each word, make every word to lower case
    params: series of strings
    return transformed series"""
    table = str.maketrans('', '', string.punctuation)
    tokens_punct = series.translate(table).lower()
    return tokens_punct


def remove_stop_words(series):
    """Remove stopwords in the series:str and makes words lower case
    param:series:str
    return: new_series:str
        """
    new_series =  ' '.join([word for word in series.split() if word not in stop_words])
    return new_series


def count_paragraph_sentences(series):
    """Count number of sentences in the paragraph
        :param series: pandas series of text strings
        :return: int count of sentences in the string (if no dots -> count = 1)"""
    sent_count = series.count(' zdot ')
    if sent_count == 0:
        sent_count = 1
    return sent_count


def num_count(series):
    """Count number of sentences in the paragraph
        :param series: pandas series of text strings
        :return: int count of sentences in the string"""
    return series.count('znum')


def count_words(series):
    """ Count words in each string (paragraph)
    without dots and numbers
    params: series of strings
    returns: new series:int"""

    clean_sent_len = len([word for word in series.split()])
    return clean_sent_len



In [31]:
#transform to pandas -> easy to clean
data = pd.DataFrame(unique, columns=['paragraph', 'label'])
data['paragraph'] = data.paragraph.apply(replace_numbers_str)

#this one takes an eternity (lemmatiz)
data['lemmatiz'] = data.paragraph.apply(lemmatiz)
data['contains_pron'] = data.lemmatiz.apply(have_pron)
data['tokens_punct'] = data.lemmatiz.apply(remove_punctuation)

data['remove_stop_words'] = data.tokens_punct.apply(remove_stop_words)

data['sent_count'] = data.remove_stop_words.apply(count_paragraph_sentences)
data['num_count'] = data.remove_stop_words.apply(num_count)
data['clean_paragraph_len'] = data.remove_stop_words.apply(count_words)
data['not_clean_paragraph_len'] = data.paragraph.apply(count_words)
# data.drop('paragraph', axis=1, inplace=True)

data.head()

,paragraph,label,lemmatiz,contains_pron,tokens_punct,remove_stop_words,sent_count,num_count,clean_paragraph_len,not_clean_paragraph_len
0,Alternatively omit the sparkling water shake everything in a cocktail shaker with ice and serve in a rocks glass zDOT,0,alternatively omit the sparkling water shake everything in a cocktail shaker with ice and serve in a rock glass zdot,0,alternatively omit the sparkling water shake everything in a cocktail shaker with ice and serve in a rock glass zdot,alternatively omit sparkling water shake cocktail shaker ice serve rock glass zdot,1,0,12,20
1,Optional zDOT if you're making okra drizzle with olive oil salt zNUM pepper and roast at the same time for about zNUM minutes or until they start to brown zDOT,0,optional zdot if -PRON- be make okra drizzle with olive oil salt zNUM pepper and roast at the same time for about znum minute or until -PRON- start to brown zdot,1,optional zdot if pron be make okra drizzle with olive oil salt znum pepper and roast at the same time for about znum minute or until pron start to brown zdot,optional zdot pron okra drizzle olive oil salt znum pepper roast time znum minute pron start brown zdot,1,2,18,30
2,also good on pasta in salads etc zDOT,0,also good on pasta in salad etc zdot,0,also good on pasta in salad etc zdot,good pasta salad etc zdot,1,0,5,8
3,optional I served mine on top of a few handfuls of arugula zDOT,0,optional -PRON- serve -PRON- on top of a few handful of arugula zDOT,1,optional pron serve pron on top of a few handful of arugula zdot,optional pron serve pron handful arugula zdot,1,0,7,13
4,optional step Mix your yogurt sauce together and pop it in the fridge zDOT,0,optional step mix -PRON- yogurt sauce together and pop -PRON- in the fridge zDOT,1,optional step mix pron yogurt sauce together and pop pron in the fridge zdot,optional step mix pron yogurt sauce pop pron fridge zdot,1,0,10,14


In [32]:
data[5000:5030] #5020

,paragraph,label,lemmatiz,contains_pron,tokens_punct,remove_stop_words,sent_count,num_count,clean_paragraph_len,not_clean_paragraph_len


In [33]:

data[4171:4180]

,paragraph,label,lemmatiz,contains_pron,tokens_punct,remove_stop_words,sent_count,num_count,clean_paragraph_len,not_clean_paragraph_len
4171,Stir the dry into the wet don't over mix zDOT,0,stir the dry into the wet do not over mix zdot,0,stir the dry into the wet do not over mix zdot,stir dry wet mix zdot,1,0,5,10
4172,Stir the flaxseed mixture then add the almond milk maple syrup coconut oil and apple cider vinegar zDOT,0,stir the flaxseed mixture then add the almond milk maple syrup coconut oil and apple cider vinegar zDOT,0,stir the flaxseed mixture then add the almond milk maple syrup coconut oil and apple cider vinegar zdot,stir flaxseed mixture add almond milk maple syrup coconut oil apple cider vinegar zdot,1,0,14,18
4173,Stir the greens into the pan and cook until lightly wilted zDOT Add the pasta roasted cauliflower and vinegar mixture and stir to coat zDOT Stir in zNUM to zNUM cup of the pasta water to create a light sauce zDOT Add the hazelnuts raisins and parsley zDOT,0,stir the green into the pan and cook until lightly wilt zDOT add the pasta roast cauliflower and vinegar mixture and stir to coat zdot Stir in znum to znum cup of the pasta water to create a light sauce zdot add the hazelnut raisin and parsley zDOT,0,stir the green into the pan and cook until lightly wilt zdot add the pasta roast cauliflower and vinegar mixture and stir to coat zdot stir in znum to znum cup of the pasta water to create a light sauce zdot add the hazelnut raisin and parsley zdot,stir green pan cook lightly wilt zdot add pasta roast cauliflower vinegar mixture stir coat zdot stir znum znum cup pasta water create light sauce zdot add hazelnut raisin parsley zdot,3,2,31,48
4174,Stir the sweet potatoes and broth mixture into the saut zNUM ed onion zDOT Increase the heat to high and bring the mixture to a boil zDOT Lower the heat to medium add zNUM teaspoon of salt and simmer uncovered and stirring occasionally for about zNUM minutes until the sweet potato is just tender zDOT,0,stir the sweet potato and broth mixture into the saut zNUM ed onion zdot increase the heat to high and bring the mixture to a boil zdot lower the heat to medium add znum teaspoon of salt and simmer uncover and stir occasionally for about znum minute until the sweet potato be just tender zdot,0,stir the sweet potato and broth mixture into the saut znum ed onion zdot increase the heat to high and bring the mixture to a boil zdot lower the heat to medium add znum teaspoon of salt and simmer uncover and stir occasionally for about znum minute until the sweet potato be just tender zdot,stir sweet potato broth mixture saut znum ed onion zdot increase heat high bring mixture boil zdot lower heat medium add znum teaspoon salt simmer uncover stir occasionally znum minute sweet potato tender zdot,2,3,34,55
4175,Stir the wet ingredients into the dry ingredients zDOT Stir in the oats raisins and walnuts if using folding into a tight batter zDOT Set the dough aside for zNUM minutes while the oven preheats zDOT Note zDOT if your dough seems too wet to become scoop able chill it in the fridge for this zNUM minutes and it'll firm up zDOT If your dough is too crumbly stir in zNUM to zNUM tablespoons water zDOT,0,stir the wet ingredient into the dry ingredient zDOT Stir in the oats raisin and walnut if use fold into a tight batter zDOT set the dough aside for znum minute while the oven preheat zDOT Note zdot if -PRON- dough seem too wet to become scoop able chill -PRON- in the fridge for this znum minute and -PRON- will firm up zdot if -PRON- dough be too crumbly stir in znum to zNUM tablespoon water zDOT,1,stir the wet ingredient into the dry ingredient zdot stir in the oats raisin and walnut if use fold into a tight batter zdot set the dough aside for znum minute while the oven preheat zdot note zdot if pron dough seem too wet to become scoop able chill pron in the fridge for this znum minute and pron will firm up zdot if pron dough be too crumbly stir in znum to znum tablespoon w

In [34]:
df = data[['remove_stop_words','sent_count','num_count','clean_paragraph_len','contains_pron','label']]

df

,remove_stop_words,sent_count,num_count,clean_paragraph_len,contains_pron,label
0,alternatively omit sparkling water shake cocktail shaker ice serve rock glass zdot,1,0,12,0,0
1,optional zdot pron okra drizzle olive oil salt znum pepper roast time znum minute pron start brown zdot,1,2,18,1,0
2,good pasta salad etc zdot,1,0,5,0,0
3,optional pron serve pron handful arugula zdot,1,0,7,1,0
4,optional step mix pron yogurt sauce pop pron fridge zdot,1,0,10,1,0
...,...,...,...,...,...,...
4866,znum cup toast walnut pine nut znum small clove garlic znum cup coarsely chop carrot preferably organic big handful basil leave znum cup juice znum small lemon sea salt znum freshly ground pepper znum znum cup olive oil optional zdot pinch red pepper flake optional zdot znum teaspoon caper optional zdot znum parmesan cheese,3,10,54,0,1
4867,znum cup milk greek yogurt znum tablespoon extra virgin olive oil znum tablespoon lemon juice znum teaspoon dijon mustard znum garlic clove mince znum teaspoon caper mince freshly ground black pepper,1,6,31,0,1
4868,znum cup roll oats znum cup chop pecans znum cup flour kind zdot spelt oat wheat purpose gf blend znum cup brown sugar znum teaspoon cinnamon pinch salt znum tablespoon butter vegan butter harden coconut oil plus skillet znum peach slice znum plum slice,1,8,44,0,1
4869,znum cup good quality coffee bean buy pron pre grind zdot znum cup filter water almond breeze almondmilk original znum cup latte,1,3,22,1,1


In [36]:
def save_data_to_pkl(json_file, file):
    """The function receive  file
     dumps into the pkl file
    :return full path of saved file"""
    pickle.dump(json_file, open(file,'wb'), pickle.HIGHEST_PROTOCOL)
    return file


path_to_data = save_data_to_pkl(df, '/Users/karlestermann/PycharmProjects/ai_assignment_aidock/data/new_one_page_data_clean_v1.pkl' )
print(path_to_data)




/Users/karlestermann/PycharmProjects/ai_assignment_aidock/data/new_one_page_data_clean_v1.pkl
